In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from xgboost import plot_importance
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import column_stack

In [2]:
df_loan_table = pd.read_csv('loan_table.csv', header = 0, sep=',', quotechar='"')

In [3]:
# df_loan_table

In [4]:
df_borrow_table = pd.read_csv('borrower_table.csv', header = 0, sep=',', quotechar='"')

In [5]:
# df_borrow_table

In [6]:
df_merged = df_borrow_table.merge(df_loan_table, on='loan_id', how='left')

In [7]:
#only look at cases when loan_granted = 1 else NA
df_merged = df_merged[df_merged['loan_granted'] == 1]

In [8]:
#change loan repaid to sep out and drop that and granted(all 1)
df_y = df_merged['loan_repaid']
df_x = df_merged.drop(columns=['loan_repaid', 'loan_granted'])
print(df_x.shape)
print(df_y.shape)
# for col in df_x.columns:
#     print(col)

(47654, 14)
(47654,)


In [9]:
#change all to np.nan for XGB implicit missing value handling
df_x[df_x == float('nan')] = np.nan

In [10]:
# gcc = 0
# ngc = 0
# for index, row in df_x.iterrows():
#     if row['loan_granted'] == 1:
#         gcc +=1
#     elif row['loan_granted'] == 0:
#         ngc +=1
# print(gcc)
# print(ngc)
# print(gcc +ngc)

In [11]:
# df_x['Year'] = df_x.date.str[:4]
#dont need year all 2012
df_x['Month'] = df_x.date.str[5:7]
df_x['Day'] = df_x.date.str[8:10]
df_x = df_x.drop(columns=['date'])

In [12]:
for col in df_x.columns:
    print(col)

loan_id
is_first_loan
fully_repaid_previous_loans
currently_repaying_other_loans
total_credit_card_limit
avg_percentage_credit_card_limit_used_last_year
saving_amount
checking_amount
is_employed
yearly_salary
age
dependent_number
loan_purpose
Month
Day


In [13]:
#try OHEing as well, might lead to more precise results
#logically since day/month are ordinal they shouldn't have to be one
#hot encoded as they should share a relationship(e.g. month 01-12) but 
#the relationship may be hard for XGB model to understand... so test w/ both


In [14]:
#OHE Month/Day to test post mid proj check here.

In [15]:
#check which input features are string types/non-integers so I know which ones I need
#to one hot encode for use in XGB
print("is_first_loan", type(df_x.is_first_loan.iloc[1]))
print("fully_repaid_previous_loans", type(df_x.fully_repaid_previous_loans.iloc[1]))
print("currently_repaying_other_loans", type(df_x.currently_repaying_other_loans.iloc[1]))
print("total_credit_card_limit", type(df_x.total_credit_card_limit.iloc[1]))
print("avg_percentage_credit_card_limit_used_last_year", type(df_x.avg_percentage_credit_card_limit_used_last_year.iloc[1]))
print("saving_amount", type(df_x.saving_amount.iloc[1]))
print("checking_amount", type(df_x.checking_amount.iloc[1]))
print("is_employed", type(df_x.is_employed.iloc[1]))
print("yearly_salary", type(df_x.yearly_salary.iloc[1]))
print("age", type(df_x.age.iloc[1]))
print("dependent_number", type(df_x.dependent_number.iloc[1]))
print("loan_purpose", type(df_x.loan_purpose.iloc[1]))
# print("loan_granted", type(df_x.loan_granted.iloc[1]))
print("Month", type(df_x.Month.iloc[1]))
print("Day", type(df_x.Day.iloc[1]))

is_first_loan <class 'numpy.int64'>
fully_repaid_previous_loans <class 'numpy.float64'>
currently_repaying_other_loans <class 'numpy.float64'>
total_credit_card_limit <class 'numpy.int64'>
avg_percentage_credit_card_limit_used_last_year <class 'numpy.float64'>
saving_amount <class 'numpy.int64'>
checking_amount <class 'numpy.int64'>
is_employed <class 'numpy.int64'>
yearly_salary <class 'numpy.int64'>
age <class 'numpy.int64'>
dependent_number <class 'numpy.int64'>
loan_purpose <class 'str'>
Month <class 'str'>
Day <class 'str'>


In [16]:
df_x['Day'] = (df_x['Day']).astype(int)
df_x['Month'] = (df_x['Month']).astype(int)

In [17]:
df_x.shape

(47654, 15)

In [18]:
df_x

,loan_id,is_first_loan,fully_repaid_previous_loans,currently_repaying_other_loans,total_credit_card_limit,avg_percentage_credit_card_limit_used_last_year,saving_amount,checking_amount,is_employed,yearly_salary,age,dependent_number,loan_purpose,Month,Day
2,135565,1,NaN,NaN,6900,0.82,2085,3422,1,24500,38,8,other,7,16
5,423171,1,NaN,NaN,6100,0.53,6163,5298,1,29500,24,1,other,11,7
7,200139,1,NaN,NaN,4000,0.57,602,2757,1,31700,36,8,business,9,19
8,991294,0,1.0,0.0,7000,0.52,2575,2917,1,58900,33,3,emergency_funds,12,4
9,875332,0,1.0,0.0,4300,0.83,722,892,1,5400,32,7,business,1,20
10,954215,1,NaN,NaN,7000,0.64,2113,5589,0,0,40,2,business,8,9
11,778076,1,NaN,NaN,4900,0.60,460,4297,1,26600,53,3,business,5,10
13,73441,0,1.0,0.0,3900,0.77,989,1547,1,45700,29,4,emergency_funds,11,8
16,249377,1,NaN,NaN,5500,0.59,1154,2731,1,9300,51,2,business,9,18
17,387504,0,1.0,0.0,5300,0.69,1172,3323,1,32500,30,1,investment,10,3


In [19]:
one_hot = pd.get_dummies(df_x['loan_purpose'])
df_x = df_x.drop(columns=['loan_purpose', 'loan_id'])
df_x = df_x.join(one_hot)


In [20]:
df_x.shape

(47654, 18)

In [21]:
df_x

,is_first_loan,fully_repaid_previous_loans,currently_repaying_other_loans,total_credit_card_limit,avg_percentage_credit_card_limit_used_last_year,saving_amount,checking_amount,is_employed,yearly_salary,age,dependent_number,Month,Day,business,emergency_funds,home,investment,other
2,1,NaN,NaN,6900,0.82,2085,3422,1,24500,38,8,7,16,0,0,0,0,1
5,1,NaN,NaN,6100,0.53,6163,5298,1,29500,24,1,11,7,0,0,0,0,1
7,1,NaN,NaN,4000,0.57,602,2757,1,31700,36,8,9,19,1,0,0,0,0
8,0,1.0,0.0,7000,0.52,2575,2917,1,58900,33,3,12,4,0,1,0,0,0
9,0,1.0,0.0,4300,0.83,722,892,1,5400,32,7,1,20,1,0,0,0,0
10,1,NaN,NaN,7000,0.64,2113,5589,0,0,40,2,8,9,1,0,0,0,0
11,1,NaN,NaN,4900,0.60,460,4297,1,26600,53,3,5,10,1,0,0,0,0
13,0,1.0,0.0,3900,0.77,989,1547,1,45700,29,4,11,8,0,1,0,0,0
16,1,NaN,NaN,5500,0.59,1154,2731,1,9300,51,2,9,18,1,0,0,0,0
17,0,1.0,0.0,5300,0.69,1172,3323,1,32500,30,1,10,3,0,0,0,1,0


In [22]:
# #OHE loan_purpose
# lp_label_encoder = LabelEncoder()
# lp_feature = lp_label_encoder.fit_transform(df_x.loan_purpose.iloc[:].values)

# #print(lp_feature)
# lp_feature = lp_feature.reshape(df_x.shape[0], 1)
# #print(lp_feature)
# onehot_encoder = OneHotEncoder(sparse = False)
# lp_feature = onehot_encoder.fit_transform(lp_feature)
# # print(type(lp_feature))
# print(df_x.loan_purpose.nunique())
# print(lp_feature.shape)

In [23]:
# #need to convert pandas series' for our already numeric features into numpy arrays so we can stack 
# #them onto the encoded_features so that all features are in one 2d array
# is_first_loan = df_x.is_first_loan.values
# fully_repaid_previous_loans = df_x.fully_repaid_previous_loans.values
# currently_repaying_other_loans = df_x.currently_repaying_other_loans.values
# total_credit_card_limit = df_x.total_credit_card_limit.values
# avg_percentage_credit_card_limit_used_last_year = df_x.avg_percentage_credit_card_limit_used_last_year.values
# saving_amount = df_x.saving_amount.values
# checking_amount = df_x.checking_amount.values
# is_employed = df_x.is_employed.values
# yearly_salary = df_x.yearly_salary.values
# age = df_x.age.values
# dependent_number = df_x.dependent_number.values
# # loan_granted = df_x.loan_granted.values
# Month = df_x.Month.values
# Day = df_x.Day.values


In [24]:
# #df_x = 16 w/ 1 for id(remove in model) and 1 for loan_purpose, remove = 14, lp_OHE'd = 5, 14+5 = 19 so it works
# columns = []
# columns.append(lp_feature)
# columns.append(is_first_loan)
# columns.append(fully_repaid_previous_loans)
# columns.append(currently_repaying_other_loans)
# columns.append(total_credit_card_limit)
# columns.append(avg_percentage_credit_card_limit_used_last_year)
# columns.append(saving_amount)
# columns.append(checking_amount)
# columns.append(is_employed)
# columns.append(yearly_salary)
# columns.append(age)
# columns.append(dependent_number)
# # columns.append(loan_granted)
# columns.append(Month)
# columns.append(Day)
# encoded_feats = column_stack(columns)
# print(encoded_feats.shape)
# print(df_x.shape)

In [25]:
#split whole dataset, but in train set only train when loan_granted=1 and drop loan_granted, in test set drop 
#loan_granted and loan_repaid, predict and change predicted values to loan_granted then check against loan_repaid
random_seed = 9
t_s = .33
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size = t_s, random_state = random_seed)

In [26]:
X_test.shape

(15726, 18)

In [27]:
# tester = df_x[0:1].fully_repaid_previous_loans
from sklearn.metrics import accuracy_score
model = XGBClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(accuracy * 100.0)

91.9750731273051


C:\Users\denni\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [28]:
#since only doing cases where loan_granted = 1, any time y_test = 0 dock a point, any time y_test = 1, add a point
bank_score = 0
cnt = 0
for value in y_test:
    if value == 1:
        bank_score += 1
        cnt += 1
    elif value == 0:
        bank_score -= 1
        cnt += 1
print("Bank score = " + str(bank_score))
    

Bank score = 4558


In [29]:
xgb_score = 0
cnt = 0
for index, value in enumerate(predictions):
    if value == 1:
        if y_test.iloc[index] == predictions[index]:
            xgb_score += 1
        else:
            xgb_score -=1
        cnt += 1
    elif value == 0:
        cnt += 1
print("XGB Initial score = " + str(xgb_score))

XGB Initial score = 8880
